Author : Erkhembayar J.

Purpose : Advanced Training for AI Drug Discovery (Educational Purpose)

Link :

Korea AI Center for Drug Discovery and Development (KAICD) 2020

# **Coding Session Environment Setting**
---

In [40]:
!pip install dgl==0.4.3
!pip install dglke

# **WHAT WILL YOU LEARN?**

<p align="center">
  <img src="https://drive.google.com/uc?id=1mQZ0EwwEL_ioYLUpuvnw02E4CLe6jM-Y" />
</p>


# 1) **UNDERSTANDING KNOWLEDGE GRAPH**

## **Data ETL (Extraction, Transform, Load)**
----

### Download DGL-KE DR Knowledge Graph Database (DRKG)





In [41]:
!wget https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz

--2020-11-24 17:54:01--  https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz
Resolving dgl-data.s3-us-west-2.amazonaws.com (dgl-data.s3-us-west-2.amazonaws.com)... 52.218.152.97
Connecting to dgl-data.s3-us-west-2.amazonaws.com (dgl-data.s3-us-west-2.amazonaws.com)|52.218.152.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216650245 (207M) [application/x-tar]
Saving to: ‘drkg.tar.gz.1’

drkg.tar.gz.1       100%[===================>] 206.61M  22.4MB/s    in 10s     

2020-11-24 17:54:12 (20.2 MB/s) - ‘drkg.tar.gz.1’ saved [216650245/216650245]



In [42]:
!ls

coronavirus-related-host-genes.tsv  drkg.tar.gz.1   infer_drug.tsv
COVID19_clinical_trial_drugs.tsv    drkg.tsv	    relation_glossary.tsv
covid19-host-genes.tsv		    embed	    sample_data
drkg.tar.gz			    entity2src.tsv


In [43]:
!tar xvzf drkg.tar.gz

._drkg.tsv
drkg.tsv
._embed
embed/
embed/DRKG_TransE_l2_relation.npy
embed/._relations.tsv
embed/relations.tsv
embed/._entities.tsv
embed/entities.tsv
embed/Readme.md
embed/mol_edgepred.npy
embed/mol_infomax.npy
embed/mol_masking.npy
embed/mol_contextpred.npy
embed/DRKG_TransE_l2_entity.npy
._entity2src.tsv
entity2src.tsv
._relation_glossary.tsv
relation_glossary.tsv


In [44]:
!ls

coronavirus-related-host-genes.tsv  drkg.tar.gz.1   infer_drug.tsv
COVID19_clinical_trial_drugs.tsv    drkg.tsv	    relation_glossary.tsv
covid19-host-genes.tsv		    embed	    sample_data
drkg.tar.gz			    entity2src.tsv


## About DRKG
---
* released by amazon (Amazon Shanhai AI Lab)
* 97,238 entities (compound , target, disease etc.)
* 5,874,261 triplets
* 107 types of relations

<p align="center">
  <img src="https://drive.google.com/uc?id=1_g9VNC1k8AXkaD6bIn0yIpV5Qh8HSse0" />
</p>



In [ ]:
!ls

drkg.tar.gz  embed	     relation_glossary.tsv
drkg.tsv     entity2src.tsv  sample_data


<p align="center">
  <h1>Knowledge Graph<h1>
  <img src="https://drive.google.com/uc?id=1lkzFHJJqMxpRLilatDgsildFRMg29N26" />
</p>
---

* CONCEPTS
    * **TRIPLETS** (ENTITY--RELATION---ENTITY) => ${h, r, t}$
        * Each triplet contains one connection (relation)
    * **RELATIONSHIP**
        * ontology of the KG


## How to read (understand) biomedical knowledge graphs ?

In [45]:
from enum import Enum, unique

## Define Triplet

@unique
class TripletNames(Enum):

    HEAD = 'HEAD'
    RELATION = 'RELATION'
    TAIL = 'TAIL'

col_names = [name.value for name in TripletNames]
col_names

['HEAD', 'RELATION', 'TAIL']

In [46]:
!ls

coronavirus-related-host-genes.tsv  drkg.tar.gz.1   infer_drug.tsv
COVID19_clinical_trial_drugs.tsv    drkg.tsv	    relation_glossary.tsv
covid19-host-genes.tsv		    embed	    sample_data
drkg.tar.gz			    entity2src.tsv


In [47]:
import pandas as pd 

_file = 'drkg.tsv'
_df = pd.read_csv('drkg.tsv', sep='\t', names= col_names)


In [48]:
_df.head()

,HEAD,RELATION,TAIL
0,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2157
1,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::5264
2,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2158
3,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::3309
4,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::28912


### **TRIPLETs**

<p align="center">
  <img src="https://drive.google.com/uc?id=1FCJRiCU-kGmf3BTwZQZyhLggO-7JgXKK" />
</p>

In [51]:
# number of triplets and relationships in training Knowledge Graph
print(f'# of triplets: {_df.shape[0]}')
print(f'# of relation: {len(_df[TripletNames.RELATION.value].unique())}')


# of triplets: 5874261
# of relation: 107


### **RELATIONs**
* Relation (ontologies) represent the backbone of the formal semantics of a knowledge graph.



In [52]:
relation_types = _df[TripletNames.RELATION.value].unique()
relation_types

array(['bioarx::HumGenHumGen:Gene:Gene', 'bioarx::VirGenHumGen:Gene:Gene',
       'bioarx::DrugVirGen:Compound:Gene',
       'bioarx::DrugHumGen:Compound:Gene',
       'bioarx::Covid2_acc_host_gene::Disease:Gene',
       'bioarx::Coronavirus_ass_host_gene::Disease:Gene',
       'DGIDB::INHIBITOR::Gene:Compound',
       'DGIDB::ANTAGONIST::Gene:Compound', 'DGIDB::OTHER::Gene:Compound',
       'DGIDB::AGONIST::Gene:Compound', 'DGIDB::BINDER::Gene:Compound',
       'DGIDB::MODULATOR::Gene:Compound', 'DGIDB::BLOCKER::Gene:Compound',
       'DGIDB::CHANNEL BLOCKER::Gene:Compound',
       'DGIDB::ANTIBODY::Gene:Compound',
       'DGIDB::POSITIVE ALLOSTERIC MODULATOR::Gene:Compound',
       'DGIDB::ALLOSTERIC MODULATOR::Gene:Compound',
       'DGIDB::ACTIVATOR::Gene:Compound',
       'DGIDB::PARTIAL AGONIST::Gene:Compound',
       'DRUGBANK::x-atc::Compound:Atc',
       'DRUGBANK::ddi-interactor-in::Compound:Compound',
       'DRUGBANK::target::Compound:Gene',
       'DRUGBANK::enzyme::Compou

### **HOW TO READ RELATIONS**

* RELATION EXAMPLE 
<p align="center">
  <img src="https://drive.google.com/uc?id=1RlAdSeoK7xrh6QXIPPKgwtJwsVjwthN1" />
</p>



<p align="center">
  <img src="https://drive.google.com/uc?id=1wVdnTzHVT2f36R_wJauIG-4VGmG6k64-" />
</p>



In [53]:
!ls

coronavirus-related-host-genes.tsv  drkg.tar.gz.1   infer_drug.tsv
COVID19_clinical_trial_drugs.tsv    drkg.tsv	    relation_glossary.tsv
covid19-host-genes.tsv		    embed	    sample_data
drkg.tar.gz			    entity2src.tsv


In [54]:
ontology = pd.read_csv('relation_glossary.tsv', sep='\t')
ontology.head()

,Relation-name,Data-source,Connected entity-types,Interaction-type,Description,Reference for the description
0,DGIDB::ACTIVATOR::Gene:Compound,DGIDB,Compound:Gene,activation,An activator interaction is when a drug activa...,http://www.dgidb.org/getting_started
1,DGIDB::AGONIST::Gene:Compound,DGIDB,Compound:Gene,agonism,An agonist interaction occurs when a drug bind...,http://www.dgidb.org/getting_started
2,DGIDB::ALLOSTERIC MODULATOR::Gene:Compound,DGIDB,Compound:Gene,allosteric modulation,An allosteric modulator interaction occurs whe...,http://www.dgidb.org/getting_started
3,DGIDB::ANTAGONIST::Gene:Compound,DGIDB,Compound:Gene,antagonism,An antagonist interaction occurs when a drug b...,http://www.dgidb.org/getting_started
4,DGIDB::ANTIBODY::Gene:Compound,DGIDB,Compound:Gene,antibody,An antibody interaction occurs when an antibod...,http://www.dgidb.org/getting_started


In [55]:
ontology['Data-source'].unique()

array(['DGIDB', 'DRUGBANK', 'GNBR', 'HETIONET', 'IntAct', 'STRING',
       'BIBLIOGRAPHY'], dtype=object)

In [56]:
ontology[ontology['Relation-name'] == 'bioarx::HumGenHumGen:Gene:Gene']


,Relation-name,Data-source,Connected entity-types,Interaction-type,Description,Reference for the description
105,bioarx::HumGenHumGen:Gene:Gene,BIBLIOGRAPHY,Gene:Gene,interaction,Protein-protein interaction,NaN


In [57]:
interaction_types = ontology['Interaction-type'].unique()
interaction_types

array(['activation', 'agonism', 'allosteric modulation', 'antagonism',
       'antibody', 'binding', 'blocking', 'channel blocking',
       'inhibition', 'modulation', 'other', 'partial agonism',
       'positive allosteric modulation', 'carrier',
       'drug-drug interaction', 'enzyme', 'target',
       'Compound belongs to Anatomical Therapeutic Chemical (ATC) code.',
       'Compound treats the disease', 'agonism, activation',
       'antagonism, blocking', 'binding, ligand (esp. receptors)',
       'inhibits cell growth (esp. cancers)', 'drug targets',
       'increases expression/production',
       'decreases expression/production',
       'affects expression/production (neutral)', 'promotes progression',
       'same protein or complex', 'signaling pathway',
       'role in disease pathogenesis', 'role in pathogenesis',
       'metabolism, pharmacokinetics',
       'improper regulation linked to disease', 'biomarkers (diagnostic)',
       'biomarkers (of disease progression)', 

In [58]:
!ls

coronavirus-related-host-genes.tsv  drkg.tar.gz.1   infer_drug.tsv
COVID19_clinical_trial_drugs.tsv    drkg.tsv	    relation_glossary.tsv
covid19-host-genes.tsv		    embed	    sample_data
drkg.tar.gz			    entity2src.tsv


# COVID19: Targets and Drugs


In [59]:
!wget https://www.dropbox.com/s/6wkwkq1702howzm/coronavirus-related-host-genes.tsv
!wget https://www.dropbox.com/s/1ic85679gj56a5t/COVID19_clinical_trial_drugs.tsv
!wget https://www.dropbox.com/s/ppgdw32bw9pyas3/covid19-host-genes.tsv
!wget https://www.dropbox.com/s/id0rmebzzopm25j/infer_drug.tsv


--2020-11-24 18:01:29--  https://www.dropbox.com/s/6wkwkq1702howzm/coronavirus-related-host-genes.tsv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6wkwkq1702howzm/coronavirus-related-host-genes.tsv [following]
--2020-11-24 18:01:29--  https://www.dropbox.com/s/raw/6wkwkq1702howzm/coronavirus-related-host-genes.tsv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc92b35b0ce1565c3aac088527bf.dl.dropboxusercontent.com/cd/0/inline/BD1thlT6C_PLsmKEMA7-tKxEZQWbqQ9EZ_fK-otexFNTiEn569WyQqLTYyozEpzMn3MYfa-1kFmfEgU9uBxGzOlonQlg28ZBpS4SQ9z2K88CPBpzEuTYVTNQXNSYe-ZK_M4/file# [following]
--2020-11-24 18:01:29--  https://uc92b35b0ce1565c3aac088527bf.dl.dropboxusercontent.com/cd/0/inline/BD1thlT6C_PLsmKEMA7-tKxEZQWbqQ9EZ_fK-otexFNTi

In [60]:
!ls

coronavirus-related-host-genes.tsv    drkg.tsv
coronavirus-related-host-genes.tsv.1  embed
COVID19_clinical_trial_drugs.tsv      entity2src.tsv
COVID19_clinical_trial_drugs.tsv.1    infer_drug.tsv
covid19-host-genes.tsv		      infer_drug.tsv.1
covid19-host-genes.tsv.1	      relation_glossary.tsv
drkg.tar.gz			      sample_data
drkg.tar.gz.1


In [76]:


import pandas as pd
import numpy as np


df = pd.read_csv('coronavirus-related-host-genes.tsv', sep="\t", names=col_names)
df_host = pd.read_csv('covid19-host-genes.tsv',  sep="\t", names=col_names)


In [77]:
df.head()

,HEAD,RELATION,TAIL
0,Disease::MESH:D001351,bioarx::Coronavirus_ass_host_gene::Disease:Gene,Gene::2931
1,Disease::MESH:D001351,bioarx::Coronavirus_ass_host_gene::Disease:Gene,Gene::2932
2,Disease::MESH:D001351,bioarx::Coronavirus_ass_host_gene::Disease:Gene,Gene::26986
3,Disease::MESH:D001351,bioarx::Coronavirus_ass_host_gene::Disease:Gene,Gene::8761
4,Disease::MESH:D001351,bioarx::Coronavirus_ass_host_gene::Disease:Gene,Gene::3178


In [78]:
df_host.head()

,HEAD,RELATION,TAIL
0,Disease::SARS-CoV2 E,bioarx::Covid2_acc_host_gene::Disease:Gene,Gene::8546
1,Disease::SARS-CoV2 E,bioarx::Covid2_acc_host_gene::Disease:Gene,Gene::23476
2,Disease::SARS-CoV2 E,bioarx::Covid2_acc_host_gene::Disease:Gene,Gene::6046
3,Disease::SARS-CoV2 E,bioarx::Covid2_acc_host_gene::Disease:Gene,Gene::10283
4,Disease::SARS-CoV2 E,bioarx::Covid2_acc_host_gene::Disease:Gene,Gene::124245


In [79]:
df_host.shape

(332, 3)

# COVID19 REPOSITIONING


1.   Define target virus entities and drug entities (e.g: fda drugs)   [DRUG-TREATMENT-VIRUS]
2.   Get pretrained Knowledge Graph embedding
3.   Predict the connection scores of all possible triplets (Drug, Treatment, Virus) using some algorithms (e.g: TransE)
4.   Rank



## COVID19 Virus Related Entities

In [81]:
## covid related virus pheno
COV_disease_list = [
'Disease::SARS-CoV2 E',
'Disease::SARS-CoV2 M',
'Disease::SARS-CoV2 N',
'Disease::SARS-CoV2 Spike',
'Disease::SARS-CoV2 nsp1',
'Disease::SARS-CoV2 nsp10',
'Disease::SARS-CoV2 nsp11',
'Disease::SARS-CoV2 nsp12',
'Disease::SARS-CoV2 nsp13',
'Disease::SARS-CoV2 nsp14',
'Disease::SARS-CoV2 nsp15',
'Disease::SARS-CoV2 nsp2',
'Disease::SARS-CoV2 nsp4',
'Disease::SARS-CoV2 nsp5',
'Disease::SARS-CoV2 nsp5_C145A',
'Disease::SARS-CoV2 nsp6',
'Disease::SARS-CoV2 nsp7',
'Disease::SARS-CoV2 nsp8',
'Disease::SARS-CoV2 nsp9',
'Disease::SARS-CoV2 orf10',
'Disease::SARS-CoV2 orf3a',
'Disease::SARS-CoV2 orf3b',
'Disease::SARS-CoV2 orf6',
'Disease::SARS-CoV2 orf7a',
'Disease::SARS-CoV2 orf8',
'Disease::SARS-CoV2 orf9b',
'Disease::SARS-CoV2 orf9c',
'Disease::MESH:D045169',
'Disease::MESH:D045473',
'Disease::MESH:D001351',
'Disease::MESH:D065207',
'Disease::MESH:D028941',
'Disease::MESH:D058957',
'Disease::MESH:D006517'
]


# treatment relations
treatment = ['GNBR::T::Compound:Disease']
# treatment = ['Hetionet::CtD::Compound:Disease','GNBR::T::Compound:Disease']

* NEW MESH TERM FOR COVID19 
<p align="center">
  <img src="https://drive.google.com/uc?id=1Rx3HcwuQRDIsk-yWiZU2ne9ZcYSR9v2d" />
</p>


In [82]:
!ls

coronavirus-related-host-genes.tsv    drkg.tsv
coronavirus-related-host-genes.tsv.1  embed
COVID19_clinical_trial_drugs.tsv      entity2src.tsv
COVID19_clinical_trial_drugs.tsv.1    infer_drug.tsv
covid19-host-genes.tsv		      infer_drug.tsv.1
covid19-host-genes.tsv.1	      relation_glossary.tsv
drkg.tar.gz			      sample_data
drkg.tar.gz.1


## Drugs to Reposition (FDA approved drugs)
---

In [108]:

import csv

# Load entity file
drug_list = []
with open("./infer_drug.tsv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['drug','ids'])
    for row_val in reader:
        drug_list.append(row_val['drug'])

In [109]:

drug_list[:10]

['Compound::DB00605',
 'Compound::DB00983',
 'Compound::DB01240',
 'Compound::DB11755',
 'Compound::DB12184',
 'Compound::DB00404',
 'Compound::DB01223',
 'Compound::DB00572',
 'Compound::DB00669',
 'Compound::DB00494']

## Treatment Relation

In [105]:
entity_idmap_file = 'embed/entities.tsv'
relation_idmap_file = 'embed/relations.tsv'

In [106]:
# Get drugname/disease name to entity ID mappings
entity_map = {}
entity_id_map = {}
relation_map = {}
with open(entity_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        entity_map[row_val['name']] = int(row_val['id'])
        entity_id_map[int(row_val['id'])] = row_val['name']
        
with open(relation_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        relation_map[row_val['name']] = int(row_val['id'])
        
# handle the ID mapping
drug_ids = []
disease_ids = []
for drug in drug_list:
    drug_ids.append(entity_map[drug])
    
for disease in COV_disease_list:
    disease_ids.append(entity_map[disease])

treatment_rid = [relation_map[treat]  for treat in treatment]

KeyError: ignored

In [87]:
len(entity_id_map)

97238

### Get the pretrained DRKG knowledge graph embedding.

In [88]:
!ls

coronavirus-related-host-genes.tsv    drkg.tsv
coronavirus-related-host-genes.tsv.1  embed
COVID19_clinical_trial_drugs.tsv      entity2src.tsv
COVID19_clinical_trial_drugs.tsv.1    infer_drug.tsv
covid19-host-genes.tsv		      infer_drug.tsv.1
covid19-host-genes.tsv.1	      relation_glossary.tsv
drkg.tar.gz			      sample_data
drkg.tar.gz.1


In [89]:
# Load embeddings
import torch as th
entity_emb = np.load('embed/DRKG_TransE_l2_entity.npy')
rel_emb = np.load('embed/DRKG_TransE_l2_relation.npy')

drug_ids = th.tensor(drug_ids).long()
disease_ids = th.tensor(disease_ids).long()
treatment_rid = th.tensor(treatment_rid)

drug_emb = th.tensor(entity_emb[drug_ids])
treatment_embs = [th.tensor(rel_emb[rid]) for rid in treatment_rid]

In [90]:
entity_emb.shape

(97238, 400)

## Drug Repurposing 
We use following algorithm to calculate the edge score. Note, here we use logsigmiod to make all scores < 0. The larger the score is, the stronger the $h$ will have $r$ with $t$.

$\mathbf{d} = \gamma - ||\mathbf{h}+\mathbf{r}-\mathbf{t}||_{2}$

$\mathbf{score} = \log\left(\frac{1}{1+\exp(\mathbf{-d})}\right)$

When doing drug repurposing, here onlythe treatment related relations is used.

In [101]:
import torch.nn.functional as fn

gamma=12.0
def transE_l2(head, rel, tail):
    score = head + rel - tail
    return gamma - th.norm(score, p=2, dim=-1)

scores_per_disease = []
dids = []
for rid in range(len(treatment_embs)):
    treatment_emb=treatment_embs[rid]
    for disease_id in disease_ids:
        disease_emb = entity_emb[disease_id]
        score = fn.logsigmoid(transE_l2(drug_emb, treatment_emb, disease_emb))
        scores_per_disease.append(score)
        dids.append(drug_ids)
scores = th.cat(scores_per_disease)
dids = th.cat(dids)


# sorting in descending order
idx = th.flip(th.argsort(scores), dims=[0])
scores = scores[idx].numpy()
dids = dids[idx].numpy()

# output prepration
_, unique_indices = np.unique(dids, return_index=True)
topk=10
topk_indices = np.sort(unique_indices)[:topk]
proposed_dids = dids[topk_indices]
proposed_scores = scores[topk_indices]

<p align="center">
  <img src="https://drive.google.com/uc?id=1cFpOqvm9sxmszV0Bctdg1BAbJvCFkRbg" />
</p>



Now we list the pairs of in form of (drug, treat, disease, score)

We select top K relevent drugs according the edge score

In [102]:
for i in range(topk):
    drug = int(proposed_dids[i])
    score = proposed_scores[i]
    
    print("{}\t{}".format(entity_id_map[drug], score))

Compound::DB00811	-0.21416784822940826
Compound::DB00993	-0.8350892663002014
Compound::DB00635	-0.8974801898002625
Compound::DB01082	-0.9854875802993774
Compound::DB01234	-0.9984006881713867
Compound::DB00982	-1.0160715579986572
Compound::DB00563	-1.0189464092254639
Compound::DB00290	-1.0641038417816162
Compound::DB01394	-1.080674648284912
Compound::DB01222	-1.084547519683838


In [113]:
clinical_drugs_file = './COVID19_clinical_trial_drugs.tsv'
clinical_drug_map = {}
with open(clinical_drugs_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['id', 'drug_name','drug_id'])
    for row_val in reader:
        clinical_drug_map[row_val['drug_id']] = row_val['drug_name']


for i in range(topk):
    drug = entity_id_map[int(proposed_dids[i])][10:17]
    if clinical_drug_map.get(drug, None) is not None:
        score = proposed_scores[i]
        print("[{}]\t{}\t{}".format(i, clinical_drug_map[drug],score , proposed_scores[i]))


## all codes available in dglke home page.

[0]	Ribavirin	-0.21416784822940826
[4]	Dexamethasone	-0.9984006881713867
[8]	Colchicine	-1.080674648284912


#Homework Clinical Trial Drugs 
* self study of embedding algorithm
* run same code for clinical trial drugs
* use the node_featurizer and edge_featurizer for drug repositioning
